In [20]:
import pandas as pd
import time
import torch
import torch.nn as nn
import numpy as np

In [21]:
# Загрузка данных
df = pd.read_csv('data/simpsons_script_lines.csv', 
                    dtype={4: str, 5: str, 6: str},
                    low_memory=False)
phrases = df['normalized_text'].tolist()

In [22]:
# Создаем словарь символов
CHARS = set('abcdefghijklmnopqrstuvwxyz -')
INDEX_TO_CHAR = ['<PAD>'] + list(CHARS)
CHAR_TO_INDEX = {char: idx for idx, char in enumerate(INDEX_TO_CHAR)}

print(f"Размер словаря: {len(INDEX_TO_CHAR)}")
print(f"Символы: {INDEX_TO_CHAR}")

Размер словаря: 29
Символы: ['<PAD>', 'g', 'u', 'f', 'x', '-', 'w', 'y', 'a', 'z', 'h', 'c', 'i', 'b', 'd', ' ', 'r', 's', 'j', 'm', 'l', 'q', 'p', 'o', 't', 'e', 'k', 'n', 'v']


In [23]:
# Подготовка данных
MAX_LEN = 50
text_data = []

for phrase in phrases:
    if type(phrase) is str:
        # Оставляем только разрешенные символы
        clean_phrase = ''.join([c for c in phrase.lower() if c in CHARS])
        if len(clean_phrase) > 0:
            text_data.append(clean_phrase)

In [24]:
# Создаем тензор с данными
X = torch.zeros((len(text_data), MAX_LEN), dtype=torch.long)

for i, phrase in enumerate(text_data):
    for j, char in enumerate(phrase[:MAX_LEN]):
        X[i, j] = CHAR_TO_INDEX.get(char, 0)  # 0 = <PAD>
    # Заполняем оставшиеся позиции паддингом
    for j in range(len(phrase), MAX_LEN):
        X[i, j] = 0

print(f"Размерность данных: {X.shape}")

Размерность данных: torch.Size([132067, 50])


In [25]:
# Определяем устройство
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")


Используемое устройство: cpu


In [26]:
# Создаем RNN модель
class SimpsonRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(SimpsonRNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        
    def forward(self, x, hidden=None):
        batch_size = x.size(0)
        
        if hidden is None:
            hidden = torch.zeros(1, batch_size, self.hidden_dim).to(device)
        
        embedded = self.embedding(x)
        rnn_out, hidden = self.rnn(embedded, hidden)
        output = self.fc(rnn_out.contiguous().view(-1, self.hidden_dim))
        return output, hidden

In [27]:
# Параметры модели
VOCAB_SIZE = len(INDEX_TO_CHAR)
EMBED_DIM = 64
HIDDEN_DIM = 128

model = SimpsonRNN(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)  # Игнорируем паддинг
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print(f"Модель создана:")
print(f"- Размер словаря: {VOCAB_SIZE}")
print(f"- Размер эмбеддингов: {EMBED_DIM}")
print(f"- Размер скрытого состояния: {HIDDEN_DIM}")

Модель создана:
- Размер словаря: 29
- Размер эмбеддингов: 64
- Размер скрытого состояния: 128


In [28]:
# Обучение модели
def train_model(model, X, epochs=2, batch_size=100):
    model.train()
    
    for epoch in range(epochs):
        start_time = time.time()
        total_loss = 0
        num_batches = len(X) // batch_size
        
        for i in range(num_batches):
            # Берем батч
            batch = X[i*batch_size:(i+1)*batch_size].to(device)
            
            # Входные данные: все символы кроме последнего
            X_batch = batch[:, :-1]
            # Целевые данные: все символы кроме первого
            Y_batch = batch[:, 1:].contiguous().view(-1)
            
            optimizer.zero_grad()
            
            # Прямой проход
            outputs, _ = model(X_batch)
            loss = criterion(outputs, Y_batch)
            
            # Обратный проход
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / num_batches
        epoch_time = time.time() - start_time
        
        print(f"Эпоха {epoch+1}/{epochs}, Время: {epoch_time:.2f}с, Loss: {avg_loss:.4f}")
        
        # Генерируем пример после каждой эпохи
        if (epoch + 1) % 5 == 0:
            generated = generate_text(model, "homer", length=50, temperature=0.8)
            print(f"Сгенерированный текст: '{generated}'")

In [29]:
# Функция для генерации текста
def generate_text(model, start_text, length=100, temperature=1.0):
    model.eval()
    
    with torch.no_grad():
        # Преобразуем начальный текст в индексы
        chars = [c for c in start_text.lower() if c in CHAR_TO_INDEX]
        if not chars:
            chars = ['h']  # fallback
            
        input_indices = [CHAR_TO_INDEX[c] for c in chars]
        input_tensor = torch.tensor([input_indices]).to(device)
        
        hidden = None
        generated_text = chars.copy()
        
        for _ in range(length):
            # Получаем предсказания
            output, hidden = model(input_tensor, hidden)
            
            # Применяем температуру
            output = output[-1] / temperature
            probabilities = torch.softmax(output, dim=-1)
            
            # Выбираем следующий символ
            next_char_idx = torch.multinomial(probabilities, 1).item()
            
            # Пропускаем паддинг
            if next_char_idx == 0:
                continue
                
            next_char = INDEX_TO_CHAR[next_char_idx]
            generated_text.append(next_char)
            
            # Обновляем вход для следующей итерации
            input_tensor = torch.tensor([[next_char_idx]]).to(device)
        
        return ''.join(generated_text)

In [30]:
# Обучаем модель
print("Начинаем обучение...")
train_model(model, X, epochs=2, batch_size=128)


Начинаем обучение...
Эпоха 1/2, Время: 25.49с, Loss: 1.8792
Эпоха 2/2, Время: 25.73с, Loss: 1.6228


In [31]:
# Тестируем генерацию
print("\n" + "="*50)
print("ТЕСТИРОВАНИЕ ГЕНЕРАЦИИ")
print("="*50)

test_starters = ["homer", "bart", "lisa", "marge"]


for starter in test_starters:
    generated = generate_text(model, starter, length=30, temperature=0.7)
    print(f"Стартер: '{starter}' -> Генерация: '{generated}'")


ТЕСТИРОВАНИЕ ГЕНЕРАЦИИ
Стартер: 'homer' -> Генерация: 'homer hell you know the dollara i a'
Стартер: 'bart' -> Генерация: 'bart id gonna be here you hello yo'
Стартер: 'lisa' -> Генерация: 'lisa have simpson a but sorry that'
Стартер: 'marge' -> Генерация: 'marge the with miss to rip score th'
